In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
from sklearn.compose import make_column_transformer
import time
from datetime import datetime, timedelta




In [7]:
q = 0
last_trial_time = datetime.now()
while q < 1:
    
    print(last_trial_time)
    control_dataframe = pd.read_csv('control.csv' ) 
    
    if control_dataframe['predict'].iloc[:1].values == 1:     
        
        q = q+1
        
        
        
    else:  
        last_trial_time = datetime.now()
        time.sleep(60)



2021-12-09 16:21:38.862246


In [8]:
print('---------------- Script Has Started Execution-----------------------')

headers = ['date', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]

current_week_dataframe = pd.read_csv('current_week_values/current_week_values.csv' ) 

current_week_date = current_week_dataframe['date'].iloc[:1].values[0]

current_week_date = datetime.strptime(current_week_date, '%Y.%m.%d')

week_start_date  = current_week_date + timedelta(days=7)
week_end_date = current_week_date + timedelta(days=12)
week_start_date = week_start_date.strftime("%Y.%m.%d")
week_end_date = week_end_date.strftime("%Y.%m.%d")      
                                           
                                           


---------------- Script Has Started Execution-----------------------


In [9]:
trade_info = pd.read_csv('files/next_week_trades.csv' ) 
    


currency_pairs = []

for index, row in trade_info.iterrows():
    
    current_trade_info = {'name' : row.names, 'direction': row.next_week_prediction, 'dataframe': None }
    
    currency_pairs.append(current_trade_info)

print(currency_pairs)


[{'name': 'GBPUSD', 'direction': 'B', 'dataframe': None}, {'name': 'EURUSD', 'direction': 'B', 'dataframe': None}, {'name': 'USDJPY', 'direction': 'S', 'dataframe': None}, {'name': 'AUDUSD', 'direction': 'B', 'dataframe': None}, {'name': 'GBPCHF', 'direction': 'B', 'dataframe': None}, {'name': 'EURCHF', 'direction': 'S', 'dataframe': None}, {'name': 'CADCHF', 'direction': 'B', 'dataframe': None}, {'name': 'AUDCHF', 'direction': 'S', 'dataframe': None}, {'name': 'NZDCHF', 'direction': 'S', 'dataframe': None}, {'name': 'EURGBP', 'direction': 'S', 'dataframe': None}, {'name': 'GBPCAD', 'direction': 'S', 'dataframe': None}, {'name': 'EURCAD', 'direction': 'S', 'dataframe': None}, {'name': 'EURAUD', 'direction': 'S', 'dataframe': None}, {'name': 'EURNZD', 'direction': 'B', 'dataframe': None}, {'name': 'CADJPY', 'direction': 'B', 'dataframe': None}, {'name': 'AUDJPY', 'direction': 'B', 'dataframe': None}, {'name': 'NZDJPY', 'direction': 'B', 'dataframe': None}, {'name': 'AUDCAD', 'direction'

In [10]:
# this cell checks if all the currency pairs have the same number of rows

total_rows = 0

for a in currency_pairs:
    current_currency_dataframe = pd.read_csv('../../raw_30_minutes_data/' + a['name']+ str(30) + '.csv', names=headers  ) 
    
    # save current currency dataframe to memory to speedup the consecutive calls to the dataframe by saving to a variable
    #filter dataframe to only return needed rows for the week
    
    a['dataframe']= current_currency_dataframe.loc[  (current_currency_dataframe['date'] >= week_start_date)
                                                   
                                                     & 
                                                      (current_currency_dataframe['date'] <= week_end_date)
                                                                                                      ]  
    
    
    current_currency_dataframe_row_ammount = len(a['dataframe'].index)
    
    if  current_currency_dataframe_row_ammount != 240:
        
        print(a['name'], a['dataframe'])
        raise Exception("number of rows is wrong")
        
    else:
        total_rows = current_currency_dataframe_row_ammount
        
    
    if  a['dataframe']['ignore'].iloc[-1] != '23:30':
        
        print(a['name'], a['dataframe'])
        raise Exception("last row timeframe is wrong")
        
            
    if a['dataframe']['date'].iloc[-1] != week_end_date :
        
        print(a['name'], a['dataframe'])
        raise Exception("last day date is wrong")
       
    
    
   
    
    #drops the first 24 columns so as to start analysis by 2am and not 12am
    #a['dataframe'] = a['dataframe'].iloc[24: , :]
    
    #reset the index of the dataframe to refresh it
    a['dataframe'] = a['dataframe'].reset_index(drop=True, )
    
    print('----------------------------',a['name'])
    
   
   
    
    
   
print(total_rows)

---------------------------- GBPUSD
---------------------------- EURUSD
---------------------------- USDJPY
---------------------------- AUDUSD
---------------------------- GBPCHF
---------------------------- EURCHF
---------------------------- CADCHF
---------------------------- AUDCHF
---------------------------- NZDCHF
---------------------------- EURGBP
---------------------------- GBPCAD
---------------------------- EURCAD
---------------------------- EURAUD
---------------------------- EURNZD
---------------------------- CADJPY
---------------------------- AUDJPY
---------------------------- NZDJPY
---------------------------- AUDCAD
---------------------------- NZDCAD
238


In [11]:
timeframe = []
trade_values = []

for b in range(total_rows):
    
   
    # save current timefreame to timeframe list
    timeframe.append(currency_pairs[0]['dataframe'].loc[b]['date'] + ' '+ currency_pairs[0]['dataframe'].loc[b]['ignore']  )
    
    sum_of_all_open_trades = 0
    
    for c in currency_pairs:
        
        
        if c['direction'] == 'B':
            
            current_trade_value =  c['dataframe'].loc[b]['close'] - c['dataframe'].loc[0]['open']
            
        if c['direction'] == 'S':
            
            current_trade_value =  c['dataframe'].loc[0]['open'] -  c['dataframe'].loc[b]['close']
            
        
        pips_corrector = 100000

        if c['name'].endswith('JPY'):   # use this to correct multiplier of jpy pairs
            pips_corrector = 1000
        
        current_trade_value = current_trade_value * pips_corrector
        
        #print(c['name'], '---', current_trade_value )
        
        
        
        sum_of_all_open_trades = sum_of_all_open_trades + current_trade_value
        
       
        
    
    trade_values.append(sum_of_all_open_trades)
  
    


trade_values_dataframe = pd.DataFrame()
trade_values_dataframe['timeframe'] = timeframe




trade_values_dataframe['profit'] = trade_values

print (trade_values_dataframe)

trade_values_dataframe = trade_values_dataframe.sort_values(by=['timeframe'], ascending=False,  ignore_index= True)



maximal_drawup_dataframe = pd.DataFrame()
maximal_drawup_dataframe[0] = ['' for r in trade_values]
maximal_drawup_dataframe['timeframe1'] = timeframe
maximal_drawup_dataframe['drawup'] = trade_values
maximal_drawup_dataframe.sort_values(by=['drawup'], ascending=False, inplace=True, ignore_index= True)

maximal_drawdown_dataframe = pd.DataFrame()
maximal_drawdown_dataframe[1] = ['' for r in trade_values]
maximal_drawdown_dataframe['timeframe2'] = timeframe
maximal_drawdown_dataframe['drawdown'] = trade_values
maximal_drawdown_dataframe.sort_values(by=['drawdown'], ascending=True, inplace=True, ignore_index= True)



main_dataframe = pd.concat([trade_values_dataframe, maximal_drawup_dataframe, maximal_drawdown_dataframe ], axis = 1)



main_dataframe




            timeframe   profit
0     2021.11.01 0:00    242.0
1     2021.11.01 0:30    -82.0
2     2021.11.01 1:00     31.0
3     2021.11.01 1:30    -84.0
4     2021.11.01 2:00    178.0
..                ...      ...
233  2021.11.05 20:30 -10007.0
234  2021.11.05 21:00  -9739.0
235  2021.11.05 21:30  -9627.0
236  2021.11.05 22:00  -9857.0
237  2021.11.05 22:30  -9799.0

[238 rows x 2 columns]


,timeframe,profit,0,timeframe1,drawup,1,timeframe2,drawdown
0,2021.11.05 9:30,-8730.0,,2021.11.01 17:00,742.0,,2021.11.05 13:00,-10432.0
1,2021.11.05 9:00,-9121.0,,2021.11.01 17:30,639.0,,2021.11.05 11:30,-10428.0
2,2021.11.05 8:30,-9141.0,,2021.11.01 18:30,256.0,,2021.11.05 12:00,-10240.0
3,2021.11.05 8:00,-9255.0,,2021.11.01 0:00,242.0,,2021.11.05 13:30,-10137.0
4,2021.11.05 7:30,-9237.0,,2021.11.01 16:30,242.0,,2021.11.05 12:30,-10129.0
...,...,...,...,...,...,...,...,...
233,2021.11.01 11:00,-1406.0,,2021.11.05 12:30,-10129.0,,2021.11.01 16:30,242.0
234,2021.11.01 10:30,-1607.0,,2021.11.05 13:30,-10137.0,,2021.11.01 0:00,242.0
235,2021.11.01 10:00,-1101.0,,2021.11.05 12:00,-10240.0,,2021.11.01 18:30,256.0
236,2021.11.01 0:30,-82.0,,2021.11.05 11:30,-10428.0,,2021.11.01 17:30,639.0


In [12]:
main_dataframe.to_csv('files/' + 'maximaldraw_down_and_up' + '.csv', index=False)


In [13]:
control_dataframe = pd.read_csv('control.csv' ) 
control_dataframe['drawdownup'].iloc[:1] = 1


print(control_dataframe)
control_dataframe.to_csv('control.csv', index=False)


   classify  corelation  feature  predict  drawdownup
0         1           1        1        1           1
